In [ ]:
!pip install openai pinecone-client sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.0 MB/s eta 0:00:00


In [ ]:
import openai
import pinecone

#api keys to be added here
openai.api_key = ''
pinecone.init(api_key='' , environment='')


In [ ]:
index_name = 'qa-bot-index'

if index_name not in pinecone.list_indexes():
    pinecone.create_index(index_name, dimension=768)
index = pinecone.Index(index_name)


In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

documents = [
    "What is the return policy?",
    "How can I reset my password?",
    "What are your business hours?"
]

document_embeddings = model.encode(documents)
for i, embedding in enumerate(document_embeddings):
    index.upsert([(f'doc_{i}', embedding.tolist())])


In [ ]:
def retrieve_documents(query, top_k=3):
    query_embedding = model.encode([query])[0]
    result = index.query(query_embedding.tolist(), top_k=top_k, include_values=True)
    retrieved_docs = [match['id'] for match in result['matches']]
    return retrieved_docs


In [ ]:
def generate_answer(query, documents):
    augmented_query = query + "\n\nRelevant Information:\n" + "\n".join(documents)
    response = openai.Completion.create(
        engine='text-davinci-003',
        prompt=augmented_query,
        max_tokens=150
    )
    return response.choices[0].text.strip()


In [ ]:
def rag_pipeline(query):
    retrieved_docs = retrieve_documents(query)
    answer = generate_answer(query, retrieved_docs)
    return answer


In [ ]:
query = "How do I reset my password?"
answer = rag_pipeline(query)
print("Answer:", answer)